In [ ]:
import json
import pickle
import datetime
import numpy as np
import pandas as pd
from graphqlclient import GraphQLClient
from pycoingecko import CoinGeckoAPI
import matplotlib.pyplot as plt
from iteration_utilities import duplicates

### functions & variables

In [ ]:
op_total = 135000
cutoff_timestamp = 1653339960
ovm1_timestamp = 1636696800
lp_percent = 0.69
taker_percent = 0.31
lp_scalar = 2.25
taker_scalar = 1.5
volume_cutoff = 10
pool_split = {'ETH': .325, 'WBTC': .1, 'USDC': .275, 'USDT': .1, 'DAI': .15, 'SNX': .05}

In [ ]:
# this is a function to handle the calling of data from the graph 
def graph_query(api, query, variables, column, id = 'id'):

    # set the graph client
    client = GraphQLClient(api)

    # set variables to get column values to populate dataframe
    col_variables = {"lastID": "", "amount": 1, "lastTimestamp": 0}

    # query the subgarph and load into dataframe
    col_result = client.execute(query, col_variables)
    col_data = json.loads(col_result)
    print(col_data)
    col_df = pd.json_normalize(col_data['data'][column])

    # create and empty dataframe to append to
    columns = list(col_df.columns)
    df = pd.DataFrame(columns=columns)

    # now query the subgraph for the next batch of data
    result = client.execute(query, variables)
    data = json.loads(result)
    update = pd.json_normalize(data['data'][column])

    while update.empty != True:
        df = df.append(update)
        variables['lastID'] = update[id].iloc[-1]
        result = client.execute(query, variables)
        data = json.loads(result)
        update = pd.json_normalize(data['data'][column]) 

    # reset the index
    df.reset_index(inplace = True, drop = True)

    return df

In [ ]:
# a dictionary to map asset addresses to decimals
op_main_asset_decimals = {
    '0x4200000000000000000000000000000000000006' : 18,
    '0x7f5c764cbc14f9669b88837ca1490cca17c31607' : 6,
    '0x68f180fcce6836688e9084f035309e29bf0a2095' : 8,
    '0xda10009cbd5d07dd0cecc66161fc93d7c9000da1' : 18,
    '0x94b008aa00579c1307b0ef2c499ad98a8ce58e58' : 6,
    '0x8700daec35af8ff88c16bdf0418774cb3d7599b4' : 18,
    '0x4200000000000000000000000000000000000042' : 18
}
    
# a dictionary to map asset addresses to symbols
op_main_asset_names = {
    '0x4200000000000000000000000000000000000006' : 'ETH',
    '0x7f5c764cbc14f9669b88837ca1490cca17c31607' : 'USDC',
    '0x68f180fcce6836688e9084f035309e29bf0a2095' : 'WBTC',
    '0xda10009cbd5d07dd0cecc66161fc93d7c9000da1' : 'DAI',
    '0x94b008aa00579c1307b0ef2c499ad98a8ce58e58' : 'USDT',
    '0x8700daec35af8ff88c16bdf0418774cb3d7599b4' : 'SNX',
    '0x4200000000000000000000000000000000000042' : 'OP'
}

# a dictionary to map asset addresses to symbols
op_symbols = {
    '0x4200000000000000000000000000000000000006' : 'ETH',
    '0x7f5c764cbc14f9669b88837ca1490cca17c31607' : 'USDC',
    '0x68f180fcce6836688e9084f035309e29bf0a2095' : 'WBTC',
    '0xda10009cbd5d07dd0cecc66161fc93d7c9000da1' : 'DAI',
    '0x94b008aa00579c1307b0ef2c499ad98a8ce58e58' : 'USDT',
    '0x8700daec35af8ff88c16bdf0418774cb3d7599b4' : 'SNX',
    '0x4200000000000000000000000000000000000042' : 'OP',
    '0xb0be5d911e3bd4ee2a8706cf1fac8d767a550497' : 'ETH',
    '0x7571cc9895d8e997853b1e0a1521ebd8481aa186' : 'WBTC',
    '0xe0e112e8f33d3f437d1f895cbb1a456836125952' : 'USDC',
    '0x60daec2fc9d2e0de0577a5c708bcadba1458a833' : 'DAI',
    '0xffbd695bf246c514110f5dae3fa88b8c2f42c411' : 'USDT',
    '0xeb5f29afaaa3f44eca8559c3e8173003060e919f' : 'SNX',
    '0x574a21fe5ea9666dbca804c9d69d8caf21d5322b' : 'OP'
}

In [ ]:
with open('data/pickles/exclude_list.pkl', 'rb') as f:
    exclude_list = pickle.load(f)

# LP Data Collection

### load in lp data from ovm1.0

In [ ]:
# load in the lp data from dune query: https://dune.com/queries/1134014
lp_data = pd.read_csv('data/dune/lp_ovm1.csv')

# convert the timestamp to a unix timestamp
lp_data['timestamp'] = pd.to_datetime(lp_data['timestamp'])

# convert the timestamp to a unix timestamp
lp_data['timestamp'] = lp_data.timestamp.values.astype(np.int64) // 10**9

# for strings in the asset, user, and txn columns as a 0 to the beginning of the string
lp_data['asset'] = lp_data['asset'].apply(lambda x: '0' +  x[1:])
lp_data['user'] = lp_data['user'].apply(lambda x: '0' +  x[1:])
lp_data['txn'] = lp_data['txn'].apply(lambda x: '0' +  x[1:])

# convert the asset to the symbol 
lp_data['asset'] = lp_data['asset'].apply(lambda x: op_symbols[x])

# convert change values to integers
lp_data['assetChange'] = lp_data['assetChange'].map(int)
lp_data['shareChange'] = lp_data['shareChange'].map(int)

### load in snapshot lp activity

In [ ]:
# load in the json file 
with open('data/ovm1/poolsSnapshot_ovm1.json') as f:
    data = json.loads(f.read())
    
# flatten the json data into a pandas dataframe for deposits
deposits_df =  pd.json_normalize(data, record_path='deposits')

# flatten the json data into a pandas dataframe for withdrawals
withdrawals_df =  pd.json_normalize(data, record_path='withdrawals')

# map the shares columns to integers for later calculations
deposits_df.sharesReceived = deposits_df.sharesReceived.map(int)
withdrawals_df.sharesWithdrawn = withdrawals_df.sharesWithdrawn.map(int)

# iterate through the deposits dataframe and convert the amount to decimals that is then stored in the dataframe
for index, row in deposits_df.iterrows():
    deposits_df.loc[index, 'depositedAmt'] = int(deposits_df.loc[index, 'depositedAmt']) / 10 ** (op_main_asset_decimals[row['asset']])

# iterate through the withdrawals dataframe and convert the amount to decimals that is then stored in the dataframe
for index, row in withdrawals_df.iterrows():
    withdrawals_df.loc[index, 'amountWithdrawn'] = - 1 * int(withdrawals_df.loc[index, 'amountWithdrawn']) / 10 ** (op_main_asset_decimals[row['asset']])
    withdrawals_df.loc[index, 'sharesWithdrawn'] = - 1 * int(withdrawals_df.loc[index, 'sharesWithdrawn'])

# convert the timestamp from string to integer
deposits_df['timestamp'] = deposits_df['timestamp'].map(int)
withdrawals_df['timestamp'] = withdrawals_df['timestamp'].map(int)

# convert the asset address to a symbol
deposits_df['asset'] = deposits_df['asset'].map(op_main_asset_names)
withdrawals_df['asset'] = withdrawals_df['asset'].map(op_main_asset_names)

deposits_df.rename(columns={'depositor': 'user', 'depositedAmt': 'assetChange', 'sharesReceived': 'shareChange', 'id': 'txn'}, inplace=True)
withdrawals_df.rename(columns={'withdrawer': 'user', 'amountWithdrawn': 'assetChange', 'sharesWithdrawn': 'shareChange', 'id': 'txn'}, inplace=True)

### add data from gap in snx lp activity

In [ ]:
pools = [
    '0xB0bE5d911E3BD4Ee2A8706cF1fAc8d767A550497', # Pools
    '0x7571CC9895D8E997853B1e0A1521eBd8481aa186',
    '0xe0e112e8f33d3f437D1F895cbb1A456836125952',
    '0x60daEC2Fc9d2e0de0577A5C708BcaDBA1458A833',
    '0xfFBD695bf246c514110f5DAe3Fa88B8c2f42c411',
    '0xeb5F29AfaaA3f44eca8559c3e8173003060e919f',
    '0x574a21fE5ea9666DbCA804C9d69d8Caf21d5322b'
]

# lowercase the pools
pools = [pool.lower() for pool in pools]

# create a list to store the dataframes
snx_activity = []

for file in os.listdir('data/snx/dune-gap'):

    # set a dataframe for each file
    update = pd.read_csv('data/snx/dune-gap/' + file, index_col=False)

    # rename some columns 
    update.rename(columns={'Txhash' : 'txn', 'UnixTimestamp' : 'timestamp', 'From' : 'from', 'To' : 'to' , 'TokenSymbol' : 'symbol', 'ContractAddress' : 'contract', 'Value' : 'value'}, inplace=True)

    # now filter only for lp activity, this is any transaction where either the from, to, or contract address is in the list of pools
    update = update[update['from'].isin(pools) | update.to.isin(pools) | update.contract.isin(pools)]

    # filter for only symbols that start with bath
    update = update[update.symbol.str.startswith('bath')]

    # now drop the 'bath' from the symbol
    update['symbol'] = update['symbol'].str.replace('bath', '')

    snx_activity.append(update)

snx_correction = pd.concat(snx_activity)
snx_correction.reset_index(drop=True, inplace=True)

snx_correction['value'] = snx_correction['value'].map(float)

snx_correction_df = pd.DataFrame(columns=['asset', 'user', 'assetChange', 'shareChange', 'timestamp', 'txn'])

# gap data fill 
for index, row in snx_correction.iterrows():
    # if the contract is minting bathTokens
    if row['from'] == '0x0000000000000000000000000000000000000000':
        if row['value'] == 0:
            sharec = 0
        else:
            sharec = int(snx_correction.loc[index, 'value'] * (10**18))
        snx_correction_df = snx_correction_df.append({'asset' : row['symbol'], 'user' : row['to'], 'assetChange' : row['value'], 'shareChange' : sharec, 'timestamp' : row['timestamp'], 'txn' : row['txn']}, ignore_index=True)
    # if the contract is burning bathTokens
    elif row['to'] == '0x0000000000000000000000000000000000000000':
        if row['value'] == 0:
            sharec = 0
        else:
            sharec = int(-1 * snx_correction.loc[index, 'value'] * (10**18))
        snx_correction_df = snx_correction_df.append({'asset' : row['symbol'], 'user' : row['from'], 'assetChange' : -row['value'], 'shareChange' : sharec, 'timestamp' : row['timestamp'], 'txn' : row['txn']}, ignore_index=True)
    # if users are sending bathTokens to each other
    elif (row['from'] != '0x0000000000000000000000000000000000000000') and row['to'] == '0x0000000000000000000000000000000000000000':
        if row['value'] == 0:
            sharec = 0
        else:
            sharec = int(snx_correction.loc[index, 'value'] * (10**18))
        snx_correction_df = snx_correction_df.append({'asset' : row['symbol'], 'user' : row['to'], 'assetChange' : row['value'], 'shareChange' : sharec, 'timestamp' : row['timestamp'], 'txn' : row['txn']}, ignore_index=True)
        snx_correction_df = snx_correction_df.append({'asset' : row['symbol'], 'user' : row['from'], 'assetChange' : -row['value'], 'shareChange' : (-1 * sharec), 'timestamp' : row['timestamp'], 'txn' : row['txn']}, ignore_index=True)

# set values to integers
snx_correction_df['shareChange'] = snx_correction_df['shareChange'].map(int)
snx_correction_df['timestamp'] = snx_correction_df['timestamp'].map(int)

### load in lp data from the graph

In [ ]:
# here are the queries to get the data from the graph
# define the query that is used to get the pool creation data
deposit_query = '''
    query allDeposits($lastID: ID, $amount: Int, $lastTimestamp: Int) {
        depositeds(first: $amount, where: {id_gt: $lastID, timestamp_gte: $lastTimestamp}) {
            id
            timestamp
            txn
            depositor
            asset
            depositAmount
            sharesReceived
        }
    }'''

withdraw_query = '''
    query allWithdraws($lastID: ID, $amount: Int, $lastTimestamp: Int) {
        withdraweds(first: $amount, where: {id_gt: $lastID, timestamp_gte: $lastTimestamp}) {
            id
            timestamp
            txn
            withdrawer
            asset
            withdrawAmount
            sharesBurned
        }
    }'''

In [ ]:
# now we are going to call the graph query function to get the data
api = 'https://api.thegraph.com/subgraphs/name/denverbaumgartner/lowlevellogs'
variables1 = {"lastID": "", "amount": 1000, "lastTimestamp": 0}
variables2 = {"lastID": "", "amount": 1000, "lastTimestamp": 0}
column1 = 'depositeds'
column2 = 'withdraweds'
ovm2_deposits_df = graph_query(api, deposit_query, variables1, column1, id = 'id')
ovm2_withdraws_df = graph_query(api, withdraw_query, variables2, column2, id = 'id')

#### get tranfer data from the graph 

In [ ]:
# create the query to get the data from the graph
transfer_query = '''
    query allTranferss($lastID: ID, $amount: Int, $lastTimestamp: Int, $filterString: String) {
        transfereds(first: $amount, where: {id_gt: $lastID, timestamp_gte: $lastTimestamp, recipient_not: $filterString, sender_not: $filterString}) {
            id
            timestamp
            txn
            sender
            recipient
            pool
            amount
        }
    }'''

api = 'https://api.thegraph.com/subgraphs/name/denverbaumgartner/lowlevellogs'
variables = {"lastID": "", "amount": 1000, "lastTimestamp": 0, "filterString": "0x0000000000000000000000000000000000000000"}
column = 'transfereds'
ovm2_transfers_df = graph_query(api, transfer_query, variables, column, id = 'id')


In [ ]:
# a dictionary of pool names and addresses
pool_names = {
    '0xb0be5d911e3bd4ee2a8706cf1fac8d767a550497' : 'ETH', 
    '0x7571cc9895d8e997853b1e0a1521ebd8481aa186' : 'WBTC',
    '0xe0e112e8f33d3f437d1f895cbb1a456836125952' : 'USDC', 
    '0x60daec2fc9d2e0de0577a5c708bcadba1458a833' : 'DAI',
    '0xffbd695bf246c514110f5dae3fa88b8c2f42c411' : 'USDT', 
    '0xeb5f29afaaa3f44eca8559c3e8173003060e919f' : 'SNX',
    '0x574a21fe5ea9666dbca804c9d69d8caf21d5322b' : 'OP'
}

# ['asset', 'user', 'assetChange', 'shareChange', 'timestamp', 'txn']
ovm2_transfers = pd.DataFrame(columns=['asset', 'user', 'assetChange', 'shareChange', 'timestamp', 'txn'])

# now iterate through the transfers and format to a way that could be applied to the overall dataframe
for index, row in ovm2_transfers_df.iterrows():
    ovm2_transfers = ovm2_transfers.append({'asset' : pool_names[row['pool']], 'user' : row['sender'], 'assetChange' : row['amount'], 'shareChange' : -1 * int(ovm2_transfers_df.loc[index, 'amount']), 'timestamp' : row['timestamp'], 'txn' : row['txn']}, ignore_index=True)
    ovm2_transfers = ovm2_transfers.append({'asset' : pool_names[row['pool']], 'user' : row['recipient'], 'assetChange' : row['amount'], 'shareChange' : int(ovm2_transfers_df.loc[index, 'amount']), 'timestamp' : row['timestamp'], 'txn' : row['txn']}, ignore_index=True)

# LP Data Cleaning

### clean the ovm2.0 data

In [ ]:
# convert the column values to integers
ovm2_deposits_df.depositAmount = ovm2_deposits_df.depositAmount.map(int)
ovm2_deposits_df.sharesReceived = ovm2_deposits_df.sharesReceived.map(int)
ovm2_withdraws_df.withdrawAmount = ovm2_withdraws_df.withdrawAmount.map(int)
ovm2_withdraws_df.sharesBurned = ovm2_withdraws_df.sharesBurned.map(int)

# iterate through the deposits dataframe and convert the amount to decimals that is then stored in the dataframe
for index, row in ovm2_deposits_df.iterrows():
    asset = ovm2_deposits_df.loc[index, 'asset']
    ovm2_deposits_df.loc[index, 'depositAmount'] = ovm2_deposits_df.loc[index, 'depositAmount'] / (10 ** (op_main_asset_decimals[ovm2_deposits_df.loc[index, 'asset']]))

# iterate through the withdrawals dataframe and convert the amount to decimals that is then stored in the dataframe
for index, row in ovm2_withdraws_df.iterrows():
    ovm2_withdraws_df.loc[index, 'withdrawAmount'] = ovm2_withdraws_df.loc[index, 'withdrawAmount'] / (10 ** (op_main_asset_decimals[ovm2_withdraws_df.loc[index, 'asset']]))

# convert the asset address to a symbol
ovm2_deposits_df['asset'] = ovm2_deposits_df['asset'].map(op_main_asset_names)
ovm2_withdraws_df['asset'] = ovm2_withdraws_df['asset'].map(op_main_asset_names)

### clean and merge the datasets

In [ ]:
# convert timestamp to integer
ovm2_deposits_df['timestamp'] = ovm2_deposits_df['timestamp'].map(int)
ovm2_withdraws_df['timestamp'] = ovm2_withdraws_df['timestamp'].map(int)
ovm2_transfers['timestamp'] = ovm2_transfers['timestamp'].map(int)

# convert withdrawal values to negative values
ovm2_withdraws_df['withdrawAmount'] = ovm2_withdraws_df['withdrawAmount'] * -1
ovm2_withdraws_df['sharesBurned'] = ovm2_withdraws_df['sharesBurned'] * -1


# rename the columns so they can be merged on the same columns
ovm2_deposits_df.rename(columns={'depositor': 'user', 'depositAmount': 'assetChange', 'sharesReceived': 'shareChange'}, inplace=True)
ovm2_withdraws_df.rename(columns={'withdrawer': 'user', 'withdrawAmount': 'assetChange', 'sharesBurned': 'shareChange'}, inplace=True)

merged_df = pd.DataFrame(columns = ['asset', 'user', 'assetChange', 'shareChange', 'timestamp', 'txn'])

# merge the dataframes
merged_df = merged_df.append(ovm2_deposits_df)
merged_df = merged_df.append(ovm2_withdraws_df)
merged_df = merged_df.append(lp_data)

# add in transfer data from the graph
merged_df = merged_df.append(ovm2_transfers)

# add in snapshot data 
merged_df = merged_df.append(deposits_df)
merged_df = merged_df.append(withdrawals_df)

# add in the snx data 
merged_df = merged_df.append(snx_correction_df)

# sort by timestamp
# sometimes there are multiple transactions within the same timestamp, in instances like this we need to make deposits appear before withdrawals 
merged_df.sort_values(by = ['timestamp', 'shareChange'], ascending = [True, False], inplace=True)

# drop the id column 
merged_df.drop(columns=['id'], inplace=True)

# make all user values lowercase
merged_df['user'] = merged_df['user'].str.lower()

# set the timestamp to be an integer
merged_df['timestamp'] = merged_df['timestamp'].map(int)

# if any acivity has happened after a certain timestamp, drop it 
merged_df = merged_df[merged_df['timestamp'] <= int(cutoff_timestamp)]

# clean up ovm 1.0 data
merged_df['txn'] = merged_df['txn'].apply(lambda x: x.split('-')[1] if '-' in x else x)

# drop any duplicate transactions
merged_df = merged_df.drop_duplicates(subset=['txn', 'user'], keep='first')

# reset and drop the index
merged_df.reset_index(drop=True, inplace=True)

merged_df.head()

In [ ]:
# drop any rows that have a user in the exclude_list
merged_df = merged_df[~merged_df['user'].isin(exclude_list)]

In [ ]:
# pickle the file so we have it for later if needed 
merged_df.to_pickle('data/pickles/lp_df.pkl')

In [ ]:
# load in the dataframe from the pickle file
#smerged_df = pd.read_pickle('data/pickles/lp_df.pkl')

### split lp data by asset

In [ ]:
# a dictionary that maps the assets to their corresponding subsets of the merged dataframe
assets = list(merged_df['asset'].unique())
asset_subsets = {}

# iterate through the assets and create a dictionary of dataframes
for asset in assets: 
    subset = merged_df[merged_df['asset'] == asset]

    # reset the index 
    subset.reset_index(drop=True, inplace=True)

    # create a column that is the total shareCount for the pool 
    subset['totalShareCount'] = subset['shareChange'].cumsum()

    asset_subsets[asset] = subset

In [ ]:
# a function that takes a dataframe of liquidity events and returns a dataframe representing liquidity providers relative proportion over time for a given asset
def weighted_liquidity(history): 
    # for each user, we need to get the current amount of shares that they hold at each timestamp
    users = list(history['user'].unique())
    user_shares = {}

    # iterate through the dataframe and get the subset of the users lp activities 
    for user in users: 
        subset = history[history['user'] == user]

        # create a variable to store the users share count 
        shares = 0 

        # now go through the dataframe and get the change in share count during a timestamp
        timestamps = list(subset['timestamp'].unique())

        # set a dictionary to store the share count for each timestamp
        time_state = {}

        for timestamp in timestamps: 
            # get the subset of the dataframe for the current timestamp
            timestamp_subset = subset[subset['timestamp'] == timestamp]

            # get the sum of the share change for the current timestamp
            change = timestamp_subset['shareChange'].sum()
            shares += change
            time_state[timestamp] = shares

        user_shares[user] = time_state

    # for each timestamp we want to get the total share count for the pool 
    # within some timestamps we will have multiple transactions, so we need to sum the share changes during that period and apply that change to the previous value 
    # get all the unique timestamps
    timestamps = list(history['timestamp'].unique())

    # set the initial value of the total share count to 0
    shareCount = 0 
    timeCounts = {}
    shareCounts = []
    timeSums = {}

    # iterate through the timestamps and get the sum of shareChanges at that timestamp
    for timestamp in timestamps:
        # get the subset of the dataframe that has the same timestamp
        subset = history[history['timestamp'] == timestamp]

        # get the sum of the share changes
        change = subset['shareChange'].sum()
        shareCount = shareCount + change

        # add the timestamp and the shareCount to the dictionary
        timeCounts[timestamp] = shareCount
        shareCounts.append(shareCount)

        # add the timesatmp and sum of the share changes to the dictionary
        timeSums[timestamp] = change
    
    # create a data frame that contains the timestamps and the total share count
    timestamps = list(timeCounts.keys())

    df = pd.DataFrame()
    df['timestamp'] = timestamps
    df['shareCount'] = shareCounts

    # go through each user and fill in the share count for each timestamp
    for user in users: 
        df[user] = df['timestamp'].map(user_shares[user])
    
    # forward fill the users share count for empty timestamps
    df = df.ffill()

    # get the difference between each rows timestamps
    df['time_range'] = df['timestamp'].diff(-1)
    df['time_range'] = df['time_range'].apply(lambda x: abs(x))

    # resolve any na values to 0
    df = df.fillna(0)

    return df

### get the time weighted distribution of liquidity 

In [ ]:
# for each asset, get the weighted liquidity dataframe
pools = list(asset_subsets.keys())

# create a dictionary to store the weighted liquidity dataframes
weighted_liquidity_dfs = {}

# iterate though each pool and pass to weighted_liquidity function
# this will create a dataframe that contains the users # of shares at each timestamp
for pool in pools: 
    df = weighted_liquidity(asset_subsets[pool])
    weighted_liquidity_dfs[pool] = df

# save the dictionary to a pickle file
with open('data/pickles/weighted_liquidity_dfs.pkl', 'wb') as f:
    pickle.dump(weighted_liquidity_dfs, f)

In [ ]:
# a function to take a dataframe, multiply time_period by a scalar, and compute the time weighted liquidity distribution
def lp_multiplier(df, timestamp, multiplier):

    # set the mask for the time period that we want to apply a multiplier to
    mask = (df['timestamp'] <= timestamp)

    # multiply the time_range by the multiplier
    df.loc[mask, 'time_range'] = df['time_range'] * multiplier

    # set the list of users
    users = list(df.columns)
    users.remove('timestamp')
    users.remove('shareCount')
    users.remove('time_range')

    # now go through each user, calculate their relative proportion of the pool and multiply by the time_range
    for user in users:
        df[user] = df.apply(lambda x: (x[user] / x['shareCount']) * x['time_range'], axis=1)

    return df

In [ ]:
# we want a function that will return the relative proportion of time weighted risk a user has assumed for a given asset
def lp_proportions(df):

    # set the list of users
    users = list(df.columns)
    users.remove('timestamp')
    users.remove('shareCount')
    users.remove('time_range')

    # create a dictionary to store the relative proportions of time weighted risk for each user
    proportions = {}
    totals = {}
    total = 0 

    # iterate through each user and get the sum of their relative proportions
    for user in users:
        val = df[user].sum()
        totals[user] = val
        total += val

    # get a value for the total of all users
    for user in users: 
        proportions[user] = totals[user] / total

    return proportions

### [caution] this takes a while to run :)

In [ ]:
pools = list(weighted_liquidity_dfs.keys())

# create a dictionary to store the final dictionary of relative user weightings for each pool
final_lp_distributions = {}

# create a dictionary to store the lp_multiplier dataframes
lp_multiplier_dfs = {}

# go through and for each pool apply the multiplier and get the final proportions
for pool in pools:
    df = lp_multiplier(weighted_liquidity_dfs[pool], ovm1_timestamp, lp_scalar)
    lp_multiplier_dfs[pool] = df
    final_lp_distributions[pool] = lp_proportions(df)

# save the dictionary to a pickle file
with open('data/pickles/lp_multiplier_dfs.pkl', 'wb') as f:
    pickle.dump(lp_multiplier_dfs, f)

In [ ]:
with open('data/pickles/lp_drops.pickle', 'wb') as handle:
    pickle.dump(final_lp_distributions, handle, protocol=pickle.HIGHEST_PROTOCOL)

### determine user expected rewards

In [ ]:
# calculate the total amount of rewards that each pool will receive
pool_rewards = {}

for pool in list(pool_split.keys()):
    pool_rewards[pool] = (op_total * lp_percent) * pool_split[pool]

pool_rewards

In [ ]:
# create a dictionary to store the expected token distribution for each pool
expected_distributions = {}

pools = list(final_lp_distributions.keys())

user_rewards = {}

for pool in pools:

    pool_splits = {}

    props = final_lp_distributions[pool]

    for user in list(props.keys()):
        pool_splits[user] = props[user] * pool_rewards[pool]
    
    user_rewards[pool] = pool_splits

In [ ]:
# pickle the lp user_rewards dictionary 
with open('data/pickles/final_lp_user_rewards.pickle', 'wb') as handle:
    pickle.dump(user_rewards, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Taker Volume

### load in snapshot data

In [ ]:
# read in data from the snapshot
trade_df = pd.read_json('data/ovm1/marketSnapshot_ovm1.json')

# map to integers
trade_df.take_amt = trade_df.take_amt.map(int)
trade_df.give_amt = trade_df.give_amt.map(int)

# order the dataframe by the timestamp
trade_df.sort_values(by='timestamp', ascending=True,inplace=True)

# convert the ovm1.0 dataframe columns to matching names 
trade_df.rename(columns={'taker': 'user', 'take_amt' : 'pay_amt', 'give_amt' : 'buy_amt', 'transactionHash' : 'txn'}, inplace=True)

# drop extra columns from ovm1.0 dataset 
trade_df.drop(['id', 'maker', 'pair'], axis=1, inplace=True)

# convert the trade_df datetime object to a timestamp
trade_df['timestamp'] = trade_df['timestamp'].apply(lambda x: int(x.timestamp()))

# organize the dataframe by column 
ss_taker_df = trade_df[['user', 'timestamp', 'txn', 'pay_gem', 'pay_amt', 'buy_gem', 'buy_amt']]

ss_txns = trade_df.txn.unique().tolist()

### load in dune data

In [ ]:
# [['user', 'timestamp', 'txn', 'pay_gem', 'pay_amt', 'buy_gem', 'buy_amt']]

# load in the csv downloaded from the following dune query: https://dune.com/queries/1134223
dune_taker = pd.read_csv('data/dune/taker_ovm1.csv')

# convert the timestamp to a unix timestamp
dune_taker['timestamp'] = pd.to_datetime(dune_taker['timestamp'])

# convert the timestamp to a unix timestamp
dune_taker['timestamp'] = dune_taker.timestamp.values.astype(np.int64) // 10**9

# for addresses drop the slash and add 0 to the beginning of the string
dune_taker['user'] = dune_taker['user'].apply(lambda x: '0' +  x[1:])
dune_taker['txn'] = dune_taker['txn'].apply(lambda x: '0' +  x[1:])
dune_taker['pay_gem'] = dune_taker['pay_gem'].apply(lambda x: '0' +  x[1:])
dune_taker['buy_gem'] = dune_taker['buy_gem'].apply(lambda x: '0' +  x[1:])

# convert change values to integers
dune_taker['pay_amt'] = dune_taker['pay_amt'].map(int)
dune_taker['buy_amt'] = dune_taker['buy_amt'].map(int)

### get graph data 

In [ ]:
# define the query that is used to all logTakes
taker_query = '''
query allTakes($lastID: ID, $amount: Int, $lastTimestamp: Int) {
    takers(first: $amount, where: {id_gt: $lastID, timestamp_gte: $lastTimestamp}) {
        id
        timestamp
        transaction {id}
        user {id}
        takeAsset {id}
        makeAsset {id}
        takeAmount
        makeAmount
    }
}
'''

## take_asset = input_erc20
## make_asset = target_erc20
## takeAmount = inputAmount
## makeAmount = realizedFill
swap_query = '''
query allSwaps($lastID: ID, $amount: Int, $lastTimestamp: Int) {
    swaps(first: $amount, where: {id_gt: $lastID, timestamp_gte: $lastTimestamp}) {
        id
        timestamp
        user {id}
        takeAsset {id}
        makeAsset {id}
        takeAmount
        makeAmount
  }
}
'''


In [ ]:
# now query the graph and get the data we need 
api = 'https://api.thegraph.com/subgraphs/name/denverbaumgartner/devrubiconmarketoptimism'
variables1 = {"lastID": "", "amount": 1000, "lastTimestamp": 0}
variables2 = {"lastID": "", "amount": 1000, "lastTimestamp": 0}
column1 = 'takers'
column2 = 'swaps'
takers_df = graph_query(api, taker_query, variables1, column1, id = 'id')
swaps_df = graph_query(api, swap_query, variables2, column2, id = 'id')

# convert ovm2.0 dataframe columns to matching names
takers_df.rename(columns={'takeAmount' : 'pay_amt', 'makeAmount' : 'buy_amt', 'transaction.id' : 'txn', 'user.id' : 'user', 'takeAsset.id' : 'pay_gem', 'makeAsset.id' : 'buy_gem'}, inplace=True)

# drop extra columns from ovm2.0 dataset
takers_df.drop(['id'], axis=1, inplace=True)

# convert the takers_df timestamp from string to integer
takers_df['timestamp'] = takers_df['timestamp'].map(int) 
takers_df['pay_amt'] = takers_df['pay_amt'].map(int)
takers_df['buy_amt'] = takers_df['buy_amt'].map(int)

# clean up the taker df by column 
graph_taker_df = takers_df[['user', 'timestamp', 'txn', 'pay_gem', 'pay_amt', 'buy_gem', 'buy_amt']]

### merge in taker datasets 

In [ ]:
datasets = [ss_taker_df, dune_taker, graph_taker_df]

# concat them all togehther into a single dataframe
main_taker_df = pd.concat(datasets)

# clean up the merged dataframe, reset the index, drop any potential duplicates, and sort by timestamp
main_taker_df.reset_index(drop=True, inplace=True)

# print the before and after size of the dataframe
main_taker_df.drop_duplicates(inplace=True)

main_taker_df.sort_values(by=['timestamp'], inplace=True)

# map the pay_amt and buy_amt to an integer for later calculations
main_taker_df.pay_amt = main_taker_df.pay_amt.map(int)
main_taker_df.buy_amt = main_taker_df.buy_amt.map(int)

# iterate through the taker_df and conver the pay_amt and buy_amt to decimals
for index, row in main_taker_df.iterrows():
    main_taker_df.loc[index, 'pay_amt'] = main_taker_df.loc[index, 'pay_amt'] / (10**op_main_asset_decimals[main_taker_df.loc[index, 'pay_gem']])
    main_taker_df.loc[index, 'buy_amt'] = main_taker_df.loc[index, 'buy_amt'] / (10**op_main_asset_decimals[main_taker_df.loc[index, 'buy_gem']])

# convert the asset addresses to symbols 
main_taker_df['pay_gem'] = main_taker_df['pay_gem'].map(op_main_asset_names)
main_taker_df['buy_gem'] = main_taker_df['buy_gem'].map(op_main_asset_names)

# if the pay_gem is the stable quote set USD value to pay_amt, else set it to buy_amt
main_taker_df['USD'] = main_taker_df.apply(lambda x: x.pay_amt if x.pay_gem in ['USDC', 'USDT', 'DAI'] else x.buy_amt, axis=1)

### map newest swap router txns to users

In [ ]:
# create a dictionary that maps the id (txn hash) to the user.id
txn_to_user = {}

for index, row in swaps_df.iterrows():
    txn_to_user[swaps_df.loc[index, 'id']] = swaps_df.loc[index, 'user.id']

# within the taker_df map the transaction.id to the 
for index, row in main_taker_df.iterrows():
    if row['txn'] in txn_to_user:
        main_taker_df.loc[index, 'user'] = txn_to_user[main_taker_df.loc[index, 'txn']]

### map old swap router txns back to users

In [ ]:
swaps_df = main_taker_df[main_taker_df['user'] == '0x45969104ef4561cee269b334d8cb7a99206a09e5']
txns = list(swaps_df['txn'].unique())

router_swaps = []

for file in os.listdir('data/router/'):

    update = pd.read_csv('data/router/' + file)

    router_swaps.append(update)

agg_df = pd.concat(router_swaps)
agg_df = agg_df.drop_duplicates()

# go through an map the transaction hash to the original swap user 
mapped = {}

for txn in txns: 
    try:
        mapped[txn] = list(agg_df[agg_df['Txhash'] == txn]['From'])[0]
    except: 
        print(txn)

# for all transactions that the taker is the router, map back to the original user 
# change this to be a lamda function
txnst = list(mapped.keys())

for index, row in main_taker_df.iterrows():
    if row['txn'] in txnst:
        main_taker_df.loc[index, 'user'] = mapped[main_taker_df.loc[index, 'txn']]

In [ ]:
# check that no log take is still mapped back to the old router
main_taker_df[main_taker_df['user'] == '0x45969104ef4561cee269b334d8cb7a99206a09e5'].sort_values(by=['timestamp'])

### Filter out addresses

In [ ]:
# drop any trades that have occured after the proposal was released
taker_df = main_taker_df[main_taker_df['timestamp'] <= cutoff_timestamp]

# convert the user column to lowercase
taker_df['user'] = taker_df['user'].str.lower()

# drop any rows that have a user in the exclude_list
taker_df = taker_df[~taker_df['user'].isin(exclude_list)]

# reset the index
taker_df.reset_index(inplace = True, drop = True)

### filter addresses that exchanged < $10 in total volume

In [ ]:
# filter out transactions from users that had less than $10 USD
volume_cutoff = 10
cutoff_list = []

users = list(taker_df.user.unique())

for user in users:
        user_volume = taker_df[taker_df['user'] == user].USD.sum()
        if user_volume < volume_cutoff:
            cutoff_list.append(user)

# filter out transactions from users in the cutoff_list
taker_df = taker_df[~taker_df['user'].isin(cutoff_list)]

In [ ]:
# save the taker_df to a pickle file
taker_df.to_pickle('data/pickles/taker_df.pkl')

### determine relative user proportion of volume

In [ ]:
# a function to take a dataframe and apply a multiplier for values on or before a certain timestamp
def volume_mutiplier(df, timestamp, multiplier):

    # set the mask for the time period that we want to apply a multiplier to
    mask = (df['timestamp'] <= timestamp)

    # apply the multiplier to the values in the mask
    df.loc[mask, 'USD'] = df.loc[mask, 'USD'] * multiplier

    return df

In [ ]:
# a function to find a user's proportion of total volume
def user_proportion(df): 
    user_prop = {}
    user_volumes = {}
    users = list(df.user.unique())

    # total volume 
    total_volume = df.USD.sum()

    # go through each user and calculate their cumulative volume
    for user in users:
        user_volumes[user] = df[df.user == user].USD.sum()
        user_prop[user] = user_volumes[user] / total_volume

    return user_prop

In [ ]:
# determine the relative volume of each user
df = taker_df

df = volume_mutiplier(df, ovm1_timestamp, taker_scalar)
user_prop = user_proportion(df)

In [ ]:
# determine how much op each user will get 
users = list(user_prop.keys())

taker_coins = op_total * taker_percent

taker_drops = {}

for user in users:
    taker_drops[user] = taker_coins * user_prop[user]

In [ ]:
# save the op drops to a pickle file
with open('data/pickles/taker_drops.pickle', 'wb') as handle:
    pickle.dump(taker_drops, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# open the taker_drops pickle file
with open('data/pickles/taker_drops.pickle', 'rb') as handle:
    taker_drops = pickle.load(handle)

In [ ]:
with open('data/pickles/final_lp_user_rewards.pickle', 'rb') as handle:
    lp_drops = pickle.load(handle)

In [ ]:
overall = 0 

# print out the total drops for volume, and each pool 
total_drops = sum(taker_drops.values())
overall += total_drops
print('expected takeer drop:' + str(op_total*taker_percent))
print('Total Drops: ' + str(total_drops))

for pool in list(lp_drops.keys()):
    print('expected airdrop for ' + pool + ': ' + str(op_total*lp_percent*pool_split[pool]))
    print('Airdrop for ' + pool + ': ' + str(sum(lp_drops[pool].values())))
    overall += sum(lp_drops[pool].values())

print('expected drops for all pools: ' + str(op_total))
print('Total Drops: ' + str(overall))

In [ ]:
# create a dictionary to store the users total airdrop in 
DropTopWop = {}

for key in list(lp_drops.keys()):
    for user in list(lp_drops[key].keys()):
        try:
            DropTopWop[user] += lp_drops[key][user]
        except:
            DropTopWop[user] = lp_drops[key][user]

for kiy in list(taker_drops.keys()):
    try:
        DropTopWop[kiy] += taker_drops[kiy]
    except:
        DropTopWop[kiy] = taker_drops[kiy]

#save the DropTopWop dictionary to csv
drop = pd.DataFrame(list(DropTopWop.items()))
drop.to_csv('rubicon-op-summer-airdrop.csv')

In [ ]:
sum(DropTopWop.values())

### OP Airdrop Analysis

In [ ]:
op_total = 135000
cutoff_timestamp = 1653339960
ovm1_timestamp = 1636696800
lp_percent = 0.69
taker_percent = 0.31
lp_scalar = 2.25
taker_scalar = 1.5
volume_cutoff = 10
pool_split = {'ETH': .325, 'WBTC': .1, 'USDC': .275, 'USDT': .1, 'DAI': .15, 'SNX': .05}

In [ ]:
eligible_lps = pd.read_pickle('data/pickles/lp_df.pkl')
eligble_traders = pd.read_pickle('data/pickles/taker_df.pkl')
airdrop = pd.read_csv('rubicon-op-summer-airdrop.csv')

In [ ]:
# see how much of the airdrop went to ovm 1.0 users
ovm1_eligible_lps = eligible_lps[eligible_lps['timestamp'] <= ovm1_timestamp]
ovm1_eligible_traders = eligble_traders[eligble_traders['timestamp'] <= ovm1_timestamp]

In [ ]:
# get a list of users who were eligible for the airdrop in ovm 1.0
ovm1_users = list(ovm1_eligible_lps.user.unique())
ovm1_users.extend(list(ovm1_eligible_traders.user.unique()))

# subset the airdrop dataframe to only include users who were eligible for the airdrop in ovm 1.0
ovm1_airdrop = airdrop[airdrop['0'].isin(ovm1_users)]

# rename column 0 to user and column 1 to OP
ovm1_airdrop.rename(columns={'0': 'user', '1': 'OP'}, inplace=True)
airdrop.rename(columns={'0': 'user', '1': 'OP'}, inplace=True)

In [ ]:
# print out the proportion of the airdrop that went to ovm 1.0 users, convert decimals to percentages
print('proportion of airdrop that went to eligible ovm 1.0 users: ' + str(round((ovm1_airdrop.OP.sum() / airdrop.OP.sum()), 4) * 100) + '%')